In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Primeiros Passos com a Vertex AI Gemini API & Python SDK

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_python.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Executar no Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_python.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> Ver no GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_python.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Abrir no Workbench da Vertex AI
    </a>
  </td>
</table>


## Visão Geral

### Gemini

Gemini é uma família de modelos generativos de IA desenvolvidos pelo Google DeepMind e projetados para casos de uso multimodais. A API Gemini dá acesso aos modelos Gemini Pro Vision e Gemini Pro.

### API Vertex AI Gemini

A API Vertex AI Gemini fornece uma interface unificada para interagir com modelos Gemini. Atualmente existem dois modelos disponíveis na API Gemini:

- **Modelo Gemini Pro** (`gemini-pro`): Projetado para lidar com tarefas de linguagem natural, bate-papo multivoltas de texto e código e geração de código.
- **Modelo Gemini Pro Vision** (`gemini-pro-vision`): Suporta prompts multimodais. Você pode incluir texto, imagens e vídeo em suas solicitações de prompt e obter respostas em texto ou código.

Você pode interagir com a API Gemini usando os seguintes métodos:

- Use o [Vertex AI Studio](https://cloud.google.com/generative-ai-studio) para testes rápidos e geração de comandos
- Use o SDK da Vertex AI

Este notebook se concentra no uso do **Vertex AI SDK para Python** para chamar a API Vertex AI Gemini.

Para obter mais informações, consulte a documentação [IA Generativa na Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview).

### Objetivos

Neste tutorial, você aprenderá como usar a API Vertex AI Gemini com o SDK Vertex AI para Python para interagir com os modelos Gemini Pro (`gemini-pro`) e o Gemini Pro Vision (`gemini-pro-vision`).

Você concluirá as seguintes tarefas:

- Instalar o SDK da Vertex AI para Python
- Usar a API Vertex AI Gemini para interagir com cada modelo
   - Modelo Gemini Pro (`gemini-pro`):
     - Gere texto a partir de prompts de texto
     - Explore vários recursos e opções de configuração
   - Modelo Gemini Pro Vision (`gemini-pro-vision`):
     - Gere texto a partir de imagens e prompts de texto
     - Gere texto a partir de prompts de vídeo e texto

### Custos

Este tutorial usa os seguintes componentes de Google Cloud que podem gerar custos em sua fatura:

- Vertex AI

Saiba mais sobre [preços da Vertex AI](https://cloud.google.com/vertex-ai/pricing) e use a [calculadora de preços](https://cloud.google.com/products/calculator/) para gerar uma estimativa de custo com base no uso projetado.

## Primeiros passos

### Instale a SDK da Vertex AI
**Importante:** só descomente a linha abaixo se você **não estiver** executando este laboratório no Qwiklabs

In [ ]:
# ! pip3 install --upgrade --user google-cloud-aiplatform

### **Reinicie o kernel do seu jupyter notebook** 

Como a instalação está sendo realizada com a opção `--user` é necessário reiniciar o kernel para que os novos módulos se tornem acessíveis.

**Importante:** só descomente a linha abaixo se você **não estiver** executando este laboratório no Qwiklabs

In [ ]:
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ O kernel do notebook está sendo reiniciado. Por favor aguarde este processo ser finalizado antes de continuar com os próximos passos. ⚠️</b>
</div>

### **Somente para uso no Colab - Autentique o seu ambiente de notebook** 

Caso você esteja executando este notebook no Google Colab, descomente a célula abaixo para realizar a autenticação da sua sessão de notebook com a Google Cloud Esse passo é importante **para utilização no Colab** para garantir que as chamadas a APIs de Google Cloud funcionem sem problemas.

In [ ]:
# import sys

# # Additional authentication is required for Google Colab
# if "google.colab" in sys.modules:
#     # Authenticate user to Google Cloud
#     from google.colab import auth

#     auth.authenticate_user()

### **Somente para uso no Colab - defina o projeto Google Cloud a ser utilizado** 

Caso você esteja executando este notebook no Google Colab, descomente a célula abaixo para definir qual projeto Google Cloud será utilizado pelo Colab na execução deste notebook. Senão, siga para as próximas instruções.

In [ ]:
# if "google.colab" in sys.modules:
#     # Define project information
#     PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
#     LOCATION = "us-central1"  # @param {type:"string"}

#     # Initialize Vertex AI
#     import vertexai

#     vertexai.init(project=PROJECT_ID, location=LOCATION)

### Importe as bibliotecas necessárias

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import warnings
warnings.simplefilter("ignore", UserWarning)

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Image,
    Part,
)

## Importe o modelo `Gemini 1.0 Pro`

O Gemini Pro (`gemini-1.0-pro`) ajuda na realização de tarefas utilizando linguagem natural, chats multiturno de texto e código e para a geração de código.

In [ ]:
model = GenerativeModel("gemini-1.0-pro")

### Gere textos a partir de prompts de texto

Envie um prompt de texto para o modelo. O modelo Gemini Pro (`gemini-pro`) fornece um mecanismo de resposta de streaming. Com esta abordagem, você não precisa esperar pela resposta completa; você pode começar a processar fragmentos assim que estiverem acessíveis.


In [ ]:
responses = model.generate_content("Por que o céu é azul?", stream=True)

for response in responses:
    print(response.text, end="")

#### Experimente seus próprios prompts

- Quais são os maiores desafios que o setor de saúde enfrenta?
- Quais são os últimos desenvolvimentos na indústria automotiva?
- Quais são as maiores oportunidades no setor de varejo?
- (Tente suas próprias instruções!)

In [ ]:
prompt = """Crie uma lista numerada de 10 itens. Cada item da lista deve ser uma tendência na indústria de tecnologia.

Cada tendência deve ter menos de 5 palavras.""" # tente seu próprio prompt

responses = model.generate_content(prompt, stream=True)

for response in responses:
    print(response.text, end="")

#### Parâmetros do modelo

Cada prompt enviado ao modelo inclui valores de parâmetros que controlam como o modelo gera uma resposta. O modelo pode gerar resultados diferentes para valores de parâmetros diferentes. Você pode experimentar diferentes parâmetros do modelo para ver como os resultados mudam.


In [ ]:
generation_config = GenerationConfig(
    temperature=0.9, # altere esta variável para testar temperaturas diferentes (entre 0 e 1.0)
    top_p=1.0, # altere esta variável para testar top_p diferentes (entre 0.1 e 1.0)
    top_k=32, # altere esta variável para testar top_k diferentes (entre 1 e 40)
    candidate_count=1,
    max_output_tokens=8192, # altere esta variável para testar diferentes tamanhos de respostas (entre 1 e 8192 para o Gemini-Pro)
)

responses = model.generate_content(
    "Por que o céu é azul?",
    generation_config=generation_config,
    stream=True,
)

for response in responses:
    print(response.text, end="")

### Teste os prompts de chat

O modelo Gemini Pro oferece suporte a chats com vários turnos e é ideal para tarefas de texto que exigem interações de ida e volta. Os exemplos a seguir mostram como o modelo responde durante uma conversa multiturnos.

In [ ]:
chat = model.start_chat()

prompt = """Meu nome é Luciano. Você é meu assistente pessoal. Meus filmes favoritos são Senhor dos Anéis e Hobbit.

Sugira outro filme que eu possa gostar.
"""

responses = chat.send_message(prompt, stream=True)

for response in responses:
    print(response.text, end="")

Este prompt de acompanhamento mostra como o modelo responde com base no prompt anterior:

In [ ]:
prompt = "O que estes filmes tem em comum com os filmes que eu gosto?"

responses = chat.send_message(prompt, stream=True)

for response in responses:
    print(response.text, end="")

Você também pode ter acesso ao histórico do chat:

In [ ]:
print(chat.history)

## Importe o modelo Gemini 1.0 Pro Vision

Gemini Pro Vision (`gemini-pro-vision`) é um modelo multimodal que suporta prompts multimodais. Você pode incluir texto, imagem(s) e vídeo em suas solicitações de prompt e obter respostas em texto ou código.

In [ ]:
multimodal_model = GenerativeModel("gemini-1.0-pro-vision")

### Defina algumas funções auxiliares

Defina funções auxiliares para carregar e exibir imagens.

In [ ]:
import http.client
import typing
import urllib.request

import IPython.display
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps


def display_images(
    images: typing.Iterable[Image],
    max_width: int = 600,
    max_height: int = 350,
) -> None:
    for image in images:
        pil_image = typing.cast(PIL_Image.Image, image._pil_image)
        if pil_image.mode != "RGB":
            # RGB is supported by all Jupyter environments (e.g. RGBA is not yet)
            pil_image = pil_image.convert("RGB")
        image_width, image_height = pil_image.size
        if max_width < image_width or max_height < image_height:
            # Resize to display a smaller notebook image
            pil_image = PIL_ImageOps.contain(pil_image, (max_width, max_height))
        IPython.display.display(pil_image)


def get_image_bytes_from_url(image_url: str) -> bytes:
    with urllib.request.urlopen(image_url) as response:
        response = typing.cast(http.client.HTTPResponse, response)
        image_bytes = response.read()
    return image_bytes


def load_image_from_url(image_url: str) -> Image:
    image_bytes = get_image_bytes_from_url(image_url)
    return Image.from_bytes(image_bytes)

def get_url_from_gcs(gcs_uri: str) -> str:
    # converts gcs uri to url for image display.
    url = "https://storage.googleapis.com/" + gcs_uri.replace("gs://", "").replace(" ", "%20")
    return url

def print_multimodal_prompt(contents: list):
    for content in contents:
        if isinstance(content, Image):
            display_images([content])
        elif isinstance(content, Part):
            url = get_url_from_gcs(content.file_data.file_uri)
            IPython.display.display(load_image_from_url(url))
        else:
            print(content)

### Gere texto a partir de imagens locais

Use o método `Image.load_from_file` para carregar um arquivo local como a imagem para a qual gerar texto.

In [ ]:
# Download an image from Google Cloud Storage
! gsutil cp "gs://cloud-samples-data/generative-ai/image/320px-Felis_catus-cat_on_snow.jpg" ./image.jpg

# Load from local file
image = Image.load_from_file("image.jpg")

# Prepare contents
prompt = "Descreva esta imagem"
contents = [image, prompt]

responses = multimodal_model.generate_content(contents, stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Resposta--------")
for response in responses:
    print(response.text, end="")

### Gere texto a partir de texto e imagens

#### Imagens com URIs do Cloud Storage

Se suas imagens estiverem armazenadas no [Cloud Storage](https://cloud.google.com/storage/docs), você poderá especificar o URI do Cloud Storage da imagem para incluir no prompt. Você também deve especificar o campo `mime_type`. Os tipos MIME suportados para imagens incluem `image/png` e `image/jpeg`.

Observe que o URI (não deve ser confundido com URL) de um objeto do Cloud Storage deve sempre começar com `gs://`.

In [ ]:
# Load image from Cloud Storage URI
gcs_uri = "gs://cloud-samples-data/generative-ai/image/boats.jpeg"

# Prepare contents
image = Part.from_uri(gcs_uri, mime_type="image/jpeg")
prompt = "Descreva esta cena"
contents = [image, prompt]

responses = multimodal_model.generate_content(contents, generation_config=GenerationConfig(temperature=0.0), stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Resposta--------")
for response in responses:
    print(response.text, end="")

#### Imagens com links diretos

Você também pode usar links diretos para imagens, conforme mostrado abaixo. A função auxiliar `load_image_from_url()` (que foi declarada anteriormente) converte a imagem em bytes e a retorna como um objeto Image que pode ser enviado ao `Gemini Pro Vision` junto com o prompt de texto.

In [ ]:
# Load image from Cloud Storage URI
image_url = (
    "https://storage.googleapis.com/cloud-samples-data/generative-ai/image/boats.jpeg"
)
image = load_image_from_url(image_url) # convert to bytes

# Prepare contents
prompt = "Descreva esta cena"
contents = [image, prompt]

responses = multimodal_model.generate_content(contents, generation_config=GenerationConfig(temperature=0.0), stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Resposta--------")
for response in responses:
    print(response.text, end="")

#### Combinando várias imagens e solicitações de texto em um prompt *few-shot*

Você pode enviar mais de uma imagem por vez e também colocá-las em qualquer lugar ao lado do prompt de texto.

No exemplo abaixo, a solicitação de poucas fotos é executada para que o `Gemini Pro Vision` retorne a cidade e o ponto de referência em um formato JSON específico.

In [ ]:
# Load images from Cloud Storage URI
image1_url = "https://storage.googleapis.com/github-repo/img/gemini/intro/landmark1.jpg"
image2_url = "https://storage.googleapis.com/github-repo/img/gemini/intro/landmark2.jpg"
image3_url = "https://storage.googleapis.com/github-repo/img/gemini/intro/landmark3.jpg"
image1 = load_image_from_url(image1_url)
image2 = load_image_from_url(image2_url)
image3 = load_image_from_url(image3_url)

# Prepare prompts
prompt1 = """{"cidade": "Londres", "ponto_turistico:", "Big Ben"}"""
prompt2 = """{"cidade": "Paris", "ponto_turistico:", "Torre Eiffel"}"""

# Prepare contents
contents = [image1, prompt1, image2, prompt2, image3]

responses = multimodal_model.generate_content(contents, stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Resposta--------")
for response in responses:
    print(response.text, end="")

### Gere texto a partir de um arquivo de vídeo

Especifique o URI do Cloud Storage de um vídeo a ser incluído no prompt. Você também deve especificar o campo `mime_type`. O tipo MIME suportado para vídeo inclui `video/mp4`.

In [ ]:
file_path = "github-repo/img/gemini/multimodality_usecases_overview/pixel8.mp4"
video_uri = f"gs://{file_path}"
video_url = f"https://storage.googleapis.com/{file_path}"

IPython.display.Video(video_url, width=450)

In [ ]:
prompt = """
Responda às seguintes perguntas usando apenas o vídeo:
Qual é a profissão da pessoa principal do vídeo?
Quais são as principais funcionalidades do telefone são apresentadas neste vídeo?
Em que cidade isso foi gravado?
Forneça a resposta JSON.
"""

video = Part.from_uri(video_uri, mime_type="video/mp4")
contents = [prompt, video]

responses = multimodal_model.generate_content(contents, stream=True)

for response in responses:
    print(response.text, end="")